# Product Review Classification

## Business Understanding
Our company wants a tool that will automatically classify product reviews as _positive_ or _negative_ reviews, based on the features of the review.  This will help our Product team to perform more sophisticated analyses in the future to help ensure customer satisfaction.

## Data Understanding
We have a labeled collection of 20,000 product reviews, with an equal split of positive and negative reviews. The dataset contains the following features:

 - `ProductId` Unique identifier for the product
 - `UserId` Unqiue identifier for the user
 - `ProfileName` Profile name of the user
 - `HelpfulnessNumerator` Number of users who found the review helpful
 - `HelpfulnessDenominator` Number of users who indicated whether they found the review helpful or not
 - `Time` Timestamp for the review
 - `Summary` Brief summary of the review
 - `Text` Text of the review
 - `PositiveReview` 1 if this was labeled as a positive review, 0 if it was labeled as a negative review

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [2]:
df = pd.read_csv("reviews.csv")
df.head(3)

,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Time,Summary,Text,PositiveReview
0,B002QWHJOU,A37565LZHTG1VH,C. Maltese,1,1,1305331200,Awesome!,This is a great product. My 2 year old Golden ...,1
1,B000ESLJ6C,AMUAWXDJHE4D2,angieseashore,1,1,1320710400,Was there a recipe change?,I have been drinking Pero ever since I was a l...,0
2,B004IJJQK4,AMHHNAFJ9L958,A M,0,1,1321747200,These taste so bland.,"Look, each pack contains two servings of 120 c...",0


The data has already been cleaned, so there are no missing values

In [3]:
df.isna().sum()

ProductId                 0
UserId                    0
ProfileName               0
HelpfulnessNumerator      0
HelpfulnessDenominator    0
Time                      0
Summary                   0
Text                      0
PositiveReview            0
dtype: int64

`PositiveReview` is the target, and all other columns are features

In [6]:
X = df.drop("PositiveReview", axis=1)
y = df["PositiveReview"]

## Data Preparation

First, split into train and test sets

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train.shape

(15000, 8)

Second, prepare for modeling. The following `Pipeline` prepares all data for modeling.  It one-hot encodes the `ProductId`, applies a tf-idf vectorizer to the `Summary` and `Text`, keeps the numeric columns as-is, and drops all other columns.

The following code may take up to 1 minute to run.

In [30]:
def drop_irrelevant_columns(X):
    return X.drop(["UserId", "ProfileName"], axis=1)

pipeline = Pipeline(steps=[
    ("drop_columns", FunctionTransformer(drop_irrelevant_columns, validate=False)),
    ("transform_text_columns", ColumnTransformer(transformers=[
        ("ohe", OneHotEncoder(categories="auto", handle_unknown="ignore", sparse=False), ["ProductId"]),
        ("summary-tf-idf", TfidfVectorizer(max_features=1000), "Summary"),
        ("text-tf-idf", TfidfVectorizer(max_features=1000), "Text")
        ()
    ], remainder="passthrough"))
])

X_train_transformed = pipeline.fit_transform(X_train)
X_test_transformed = pipeline.transform(X_test)

X_train_transformed.shape

(15000, 11275)

## Modeling

Fit a `RandomForestClassifier` with the best hyperparameters.  The following code may take up to 1 minute to run.

In [9]:
rfc = RandomForestClassifier(
    random_state=42,
    n_estimators=100,
    max_depth=30,
    min_samples_split=15,
    min_samples_leaf=1
)
rfc.fit(X_train_transformed, y_train)

RandomForestClassifier(max_depth=30, min_samples_split=15, random_state=42)

## Model Evaluation

We are using _accuracy_ as our metric, which is the default metric in Scikit-Learn, so it is possible to just use the built-in `.score` method

In [10]:
print("Train accuracy:", rfc.score(X_train_transformed, y_train))
print("Test accuracy:", rfc.score(X_test_transformed, y_test))

Train accuracy: 0.9826666666666667
Test accuracy: 0.913


In [9]:
print("Train confusion matrix:")
print(confusion_matrix(y_train, rfc.predict(X_train_transformed)))
print("Test confusion matrix:")
print(confusion_matrix(y_test, rfc.predict(X_test_transformed)))

Train confusion matrix:
[[7312  177]
 [  83 7428]]
Test confusion matrix:
[[2293  218]
 [ 217 2272]]


## Business Interpretation

The tuned Random Forest Classifier model appears to be somewhat overfit on the training data, but nevertheless achieves 91% accuracy on the test data.  Of the 9% of mislabeled comments, about half are false positives and half are false negatives.

Because this is a balanced dataset, 91% accuracy is a substantial improvement over a 50% baseline.  This model is ready for production use for decision support.

In [11]:
X_train.describe

<bound method NDFrame.describe of         ProductId          UserId                  ProfileName  \
5514   B008EG5ADY  A2D7X9N3IV3S7B                deeper waters   
1266   B003ZNXCE0  A1F465XE6126IS                         Dior   
5864   B0002NYNYO  A1R4FNCN368HJ8               No Ax To Grind   
15865  B000EHP62G   ASD537RN0CAK8              luvtoshopamazon   
12892  B0013AFTGG  A24CTL534C3UN4                      eshemom   
...           ...             ...                          ...   
11284  B00017LEX4  A3K5LESV135GFJ                           -E   
11964  B0000D94OR  A2WZ1IPU382OAG     S. R. Clark "S.R. Clark"   
5390   B000H1217M  A1WUE9RWYX8F1K  Pamela Hohlbein "phohlbein"   
860    B0060HVNNA  A3II2GQFQU1E4N                         Alex   
15795  B00008CQVA   AGDS7B4DRSSMF                  P. Phillips   

       HelpfulnessNumerator  HelpfulnessDenominator        Time  \
5514                      0                       0  1338249600   
1266                      6            

### 1) Data Preparation
A train-test split has already been performed.
Additionally, there is already a pipeline in place that drops some columns and converts all text columns into a numeric format for modeling.
**Your only additional data preparation task is feature scaling.**  Tree-based models like Random Forest Classifiers do not require scaling, but TensorFlow neural networks do.
There are two main strategies you can take for this task:
#### Scaling within the existing pipeline
If you are comfortable with pipelines, this is the more polished/professional route.
1. Make a new pipeline, with a `StandardScaler` as the final step.  You can nest the steps of the previous pipeline inside of this new pipeline
2. Generate a new `X_train_transformed_scaled` by calling `.fit_transform` on the new pipeline
3. Generate a new `X_test_transformed_scaled` by calling `.transform` on the new pipeline
#### Scaling after the pipeline has finished
This is a better strategy if you are not as comfortable with pipelines.
1. Instantiate a `StandardScaler` object
2. Generate a new `X_train_transformed_scaled` by calling `.fit_transform` on the scaler object, after you have called `.fit_transform` on the pipeline
3. Generate a new `X_test_transformed_scaled` by calling `.transform` on the scaler object, after you have called `.transform` on the pipeline
If you are getting stuck at this step, skip it.  The model will still be able to fit, although the performance will be worse.  Keep in mind whether or not you scaled the data in your final analysis.
### 2) Modeling
Build a neural network classifier.  Specifically, use the `keras` submodule of the `tensorflow` library to build a multi-layer perceptron model with the `Sequential` interface.
See the [`tf.keras` documentation](https://www.tensorflow.org/guide/keras/overview) for an overview on the use of `Sequential` models. See the [Keras layers documentation](https://keras.io/layers/core/) for descriptions of the `Dense` layer options.  
1. Instantiate a `Sequential` model
2. Add an input `Dense` layer.  You'll need to specify a `input_shape` = (11275,) because this is the number of features of the transformed dataset.
3. Add 2 `Dense` hidden layers.  They can have any number of units, but keep in mind that more units will require more processing power.  We recommend an initial `units` of 64 for processing power reasons.
4. Add a final `Dense` output layer.  This layer must have exactly 1 unit because we are doing a binary prediction task.
5. Compile the `Sequential` model
6. Fit the `Sequential` model on the preprocessed training data (`X_train_transformed_scaled`) with a b`batch_size` of 50 and `epochs` of 5 for processing power reasons.
### 3) Model Tuning + Feature Engineering
If you are running out of time, skip this step.
Tune the neural network model to improve performance.  This could include steps such as increasing the units, changing the activation functions, or adding regularization.
We recommend using using a `validation_split` of 0.1 to understand model performance without utilizing the test holdout set.
You can also return to the preprocessing phase, and add additional features to the model.
### 4) Model Evaluation
Choose a final `Sequential` model, add layers, and compile.  Fit the model on the preprocessed training data (`X_train_transformed_scaled`, `y_train`) and evaluate on the preprocessed testing data (`X_test_transformed_scaled`, `y_test`) using `accuracy_score`.
### 5) Technical Communication
Write a paragraph explaining whether Northwind Trading Company should switch to using your new neural network model, or continue to use the Random Forest Classifier.  Beyond a simple comparison of performance, try to take into consideration additional considerations such as:
 - Computational complexity/resource use
 - Anticipated performance on future datasets (how might the data change over time?)
 - Types of mistakes made by the two kinds of models
You can make guesses or inferences about these considerations.
**Include at least one visualization** comparing the two types of models.  Possible points of comparison could include ROC curves, colorized confusion matrices, or time needed to train.

In [34]:
X_train_transformed.shape

(15000, 11275)

In [57]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras import regularizers

model = Sequential()

model.add(Dense(units=6, activation='relu', input_shape=(X_train_transformed.shape[1],), kernel_regularizer=regularizers.l1(0.01)))
model.add(Dense(units=64, activation='relu',kernel_regularizer=regularizers.l1(0.01)))
model.add(Dense(units=64, activation='relu', kernel_regularizer=regularizers.l1(0.01)))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer="Adam", loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_transformed, y_train, validation_data=(X_test_transformed, y_test), epochs=5, batch_size=50)  

Epoch 1/5
300/300 [==============================] - 3s 12ms/step - loss: 55063.2305 - accuracy: 0.4991 - val_loss: 27525.1992 - val_accuracy: 0.4978
Epoch 2/5
300/300 [==============================] - 2s 7ms/step - loss: 4405.5068 - accuracy: 0.5022 - val_loss: 416.8366 - val_accuracy: 0.4978
Epoch 3/5
300/300 [==============================] - 2s 7ms/step - loss: 807.3218 - accuracy: 0.5029 - val_loss: 825.2225 - val_accuracy: 0.4978
Epoch 4/5
300/300 [==============================] - 2s 7ms/step - loss: 296.9929 - accuracy: 0.5006 - val_loss: 2.6408 - val_accuracy: 0.4978
Epoch 5/5
300/300 [==============================] - 2s 8ms/step - loss: 2.6403 - accuracy: 0.5007 - val_loss: 2.6377 - val_accuracy: 0.4978


Northwinds trading company would be better served using a random forest model whose accuracy, though the model was overfit, was 90% versus the neural network model which had an accuracy metric of 50%.  Depending on the resurces of the company the amount of data that could be run on each model in the future may become an issue with neural networks as the amount of calculations given the type of model could cause the model to run very slow though it won't necessarily affect its accuracy.  

In [58]:
def plot_results(model):
    fig, (ax1,ax2) = plt.subplots(1,2)

    ax1.plot(model.history.epoch, model.history.history['loss'], label='loss')
    ax1.plot(model.history.epoch, model.history.history['val_loss'], label='val_loss')
    
    ax1.legend()
    
    ax2.plot(model.history.epoch, model.history.history['recall'], label='recall')
    ax2.plot(model.history.epoch, model.history.history['val_recall'], label='val_recall')

    ax2.legend()
    
    
plot_results(model)

NameError: name 'plt' is not defined